In [1]:
import pandas as pd
import re
import nltk
import json
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.corpus import stopwords
import emoji
import unicodedata

In [33]:
df = pd.read_csv('labeled_manual.csv')
df


,full_text,case_folding,preprocessing,label
0,Sejak ada penambahan traffic light laluan perg...,sejak ada penambahan traffic light laluan perg...,tambah traffic light lalu pergi kerja kena ban...,negative
1,355 hari Presiden @prabowo Monitor Yth Bpk Pre...,355 hari presiden @prabowo monitor yth bpk pre...,presiden monitor yth bpk presiden impor singga...,negative
2,@Bambangmulyonoo Malaysia punya kilang dimaksi...,@bambangmulyonoo malaysia punya kilang dimaksi...,malaysia maksimal diindo import negri tetangga...,negative
3,@LahrianSyahilla Pertamina juga memperkuat sek...,@lahriansyahilla pertamina juga memperkuat sek...,kuat sektor hilir tingkat kapasitas efisiensi ...,positive
4,Kalau overcapacity kilang seharusnya produksi ...,kalau overcapacity kilang seharusnya produksi ...,overcapacity produksi limpah stock cukup impor...,negative
...,...,...,...,...
245,@SUARAKRISTEN1 @detikcom Kilang Dumai Pertamin...,@suarakristen1 @detikcom kilang dumai pertamin...,dumai milik kapasitas olah mentah barel dasar ...,positive
246,@tanyakanrl Aku pake azarine ijo tosca yg 45sp...,@tanyakanrl aku pake azarine ijo tosca yg 45sp...,pake azarine ijo tosca spf pake bikin kulit ku...,positive
247,@tanyakanrl Aku pakai amaterasun yg biru tua n...,@tanyakanrl aku pakai amaterasun yg biru tua n...,pakai amaterasun biru tua nder tahun kayak pak...,positive
248,Pertamina Ungkap Proyek RDMP Balikpapan Siap L...,pertamina ungkap proyek rdmp balikpapan siap l...,proyek rdmp balikpapan laku uji operasi rfcc b...,positive


In [34]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()
stop_words = set(stopwords.words("indonesian"))

custom_stopwords = {
    "nya", "sih", "saja", "aja", "kok", "sama", "pun", "lah", "dong", "deh",
    "banget", "bgt", "yg", "ya", "kan", "kah", "nah", "loh", "tuh", "nih",
    "gitu", "gituan", "begitu", "oke", "ok", "yaa", "yaaah", "yah", "iyaa",
    "iya", "engga", "nggak", "ngga", "gak", 
}

stop_words.update(custom_stopwords)


In [36]:
try:
    with open("slang.json", "r", encoding="utf-8") as f:
        abbreviation_dict = json.load(f)
except FileNotFoundError:
    print("Warning: File 'dopping.json' tidak ditemukan using empty dict.")
    abbreviation_dict = {}

# 2. Compile Regex Patterns (Dilakukan di awal agar efisien)
PATTERN_WORD_ONLY = re.compile(r'^[a-z]+$')
PATTERN_URL = re.compile(r'http\S+|www\S+')
PATTERN_USER_HASHTAG = re.compile(r'[@#]\w+')
PATTERN_NON_ALPHA = re.compile(r'[^a-z\s]')
PATTERN_REPEATED_CHAR = re.compile(r'(.)\1{2,}')

def reduce_repeated_chars(word):
    """Mengubah 'haaaalo' menjadi 'halo'"""
    return PATTERN_REPEATED_CHAR.sub(r'\1', word)

def clean_text(text):
    """Fungsi utama preprocessing"""
    
    # Cek validitas input
    if pd.isna(text) or not isinstance(text, str):
        return ""

    # --- Tahap Cleaning Awal ---
    text = text.lower()
    text = PATTERN_URL.sub(' ', text)          # Hapus URL
    text = emoji.replace_emoji(text, replace=' ') # Hapus Emoji
    text = PATTERN_USER_HASHTAG.sub(' ', text) # Hapus @username dan #hashtag
    
    # Hapus karakter non-latin/ASCII (Clean up aneh-aneh)
    text = text.encode('ascii', 'ignore').decode('ascii')
    
    # Hapus selain huruf a-z dan spasi
    text = PATTERN_NON_ALPHA.sub(' ', text)

    # --- Tahap Tokenizing & Processing ---
    words = text.split()
    cleaned_words = []

    for w in words:
        # 1. Hapus repetisi karakter
        w = reduce_repeated_chars(w)

        # 2. Normalisasi singkatan (jika ada di dict)
        w = abbreviation_dict.get(w, w)

        # 3. Validasi: Pastikan hanya huruf (redundant check, tapi aman)
        if not PATTERN_WORD_ONLY.match(w):
            continue

        # 4. Stopword Removal Tahap 1 (Sebelum stemming untuk efisiensi)
        if w in stop_words:
            continue

        # 5. Stemming
        w = stemmer.stem(w)

        # 6. Stopword Removal Tahap 2 (Cek lagi setelah di-stem)
        if w in stop_words:
            continue
            
        cleaned_words.append(w)

    return " ".join(cleaned_words)

# Contoh penggunaan
# df['cleaned_text'] = df['text_column'].apply(clean_text)

In [37]:
df["bersih"] = df["preprocessing"].apply(clean_text)
df['bersih']

0      traffic light pergi kerja kena bangun extra sb...
1      presiden monitor yth bpk presiden impor singga...
2      malaysia maksimal diindo import negri tetangga...
3      kuat sektor hilir tingkat kapasitas efisiensi ...
4      overcapacity produksi limpah stock impor harga...
                             ...                        
245    dumai milik kapasitas olah mentah barel dasar ...
246    pake azarine ijo tosca spf pake kulit kusam ce...
247    pakai amaterasun biru tua nder kayak pakai rin...
248    proyek rdmp balikpapan laku uji operasi rfcc p...
249    gaspol proyek rdmp balikpapan masuk tahap uji ...
Name: bersih, Length: 250, dtype: object

### <h1>Naive Bayes</h1>

<h1>1. BoW + Naive Bayes</h1>

In [64]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

print("\n=== PERCOBAAN 1: Bag of Words (BoW) + Naive Bayes ===")

#split data
X_train, X_test, y_train, y_test = train_test_split(df['bersih'], df['label'], test_size=0.2, random_state=42)

# Feature Extraction: BoW
bow_vectorizer = CountVectorizer()
X_train_bow = bow_vectorizer.fit_transform(X_train)
X_test_bow = bow_vectorizer.transform(X_test)

# Model: Naive Bayes
nb_model = MultinomialNB()
nb_model.fit(X_train_bow, y_train)
y_pred_nb = nb_model.predict(X_test_bow)

print("\n======================================")
print("Akurasi Bow + Naive Bayes", accuracy_score(y_test, y_pred_nb))
print("======================================")
print("Classification Report:", "\n", classification_report(y_test, y_pred_nb))


=== PERCOBAAN 1: Bag of Words (BoW) + Naive Bayes ===

Akurasi Bow + Naive Bayes 0.72
Classification Report: 
               precision    recall  f1-score   support

    negative       0.86      0.72      0.78        25
     neutral       0.67      0.29      0.40         7
    positive       0.62      0.89      0.73        18

    accuracy                           0.72        50
   macro avg       0.71      0.63      0.64        50
weighted avg       0.74      0.72      0.71        50



<h1>2. Word2Vec + Naive Bayes</h1>

In [60]:
import numpy as np
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report

print("\n=== PERCOBAAN 2: Word2Vec + Naive Bayes ===")

# --- 1. Training Word2Vec ---
# Tokenisasi kalimat menjadi list kata
sentences = [text.split() for text in df['bersih']]

# Latih model Word2Vec
# vector_size=100 (dimensi), window=5, min_count=1
w2v_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

# Fungsi untuk mengubah kalimat menjadi rata-rata vektor kata
def sent_vectorizer(sent, model):
    words = sent.split()
    # Ambil vektor untuk setiap kata yang ada di vocabulary model
    vector_list = [model.wv[word] for word in words if word in model.wv]
    
    if len(vector_list) > 0:
        # Rata-rata vektor kata
        return np.mean(vector_list, axis=0)
    else:
        # Jika tidak ada kata yang dikenal, kembalikan vektor nol
        return np.zeros(model.vector_size)

# Terapkan vectorizer ke data
X = np.array([sent_vectorizer(text, w2v_model) for text in df['bersih']])
y = df['label']

# --- 2. Klasifikasi Naive Bayes ---
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model GaussianNB (Cocok untuk data kontinu seperti Word2Vec)
nb = GaussianNB()
nb.fit(X_train, y_train)
y_pred = nb.predict(X_test)

print("\n======================================")
print("Akurasi Word2Vec + Naive Bayes", accuracy_score(y_test, y_pred))
print("======================================")
print("Classification Report:", "\n", classification_report(y_test, y_pred))


=== PERCOBAAN 2: Word2Vec + Naive Bayes ===

Akurasi Word2Vec + Naive Bayes 0.5
Classification Report: 
               precision    recall  f1-score   support

    negative       0.75      0.36      0.49        25
     neutral       0.25      0.14      0.18         7
    positive       0.44      0.83      0.58        18

    accuracy                           0.50        50
   macro avg       0.48      0.45      0.42        50
weighted avg       0.57      0.50      0.48        50



### <h1>SVM</h1>

<h1>1. TF-IDF + SVM</h1>

In [63]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

print("\n=== PERCOBAAN 2: TF-IDF + SVM ===")

#split data
X_train, X_test, y_train, y_test = train_test_split(df['bersih'], df['label'], test_size=0.2, random_state=42)

# Feature Extraction: TF-IDF
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Model: SVM
svm_model = SVC(kernel='linear')
svm_model.fit(X_train_tfidf, y_train)
y_pred_svm = svm_model.predict(X_test_tfidf)

print("\n======================================")
print("Akurasi TF-IDF + SVM", accuracy_score(y_test, y_pred_svm))
print("======================================")
print("Classification Report:", "\n", classification_report(y_test, y_pred_svm))


=== PERCOBAAN 2: TF-IDF + SVM ===

Akurasi TF-IDF + SVM 0.8
Classification Report: 
               precision    recall  f1-score   support

    negative       0.88      0.88      0.88        25
     neutral       0.75      0.43      0.55         7
    positive       0.71      0.83      0.77        18

    accuracy                           0.80        50
   macro avg       0.78      0.71      0.73        50
weighted avg       0.80      0.80      0.79        50



<h1>2. Word2Vec + SVM</h1>

In [61]:
import numpy as np
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

print("\n=== PERCOBAAN 4: Word2Vec + SVM ===")

# --- 1. Training Word2Vec ---
sentences = [text.split() for text in df['bersih']]
w2v_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

def sent_vectorizer(sent, model):
    words = sent.split()
    vector_list = [model.wv[word] for word in words if word in model.wv]
    if len(vector_list) > 0:
        return np.mean(vector_list, axis=0)
    else:
        return np.zeros(model.vector_size)

X = np.array([sent_vectorizer(text, w2v_model) for text in df['bersih']])
y = df['label']

# --- 2. Klasifikasi SVM ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model SVM
svm = SVC(kernel='linear') # Bisa diganti 'rbf'
svm.fit(X_train, y_train)
y_pred = svm.predict(X_test)

# Evaluasi
print("\n======================================")
print("Akurasi Word2Vec + SVM", accuracy_score(y_test, y_pred))
print("======================================")
print("Classification Report:", "\n", classification_report(y_test, y_pred))


=== PERCOBAAN 4: Word2Vec + SVM ===

Akurasi Word2Vec + SVM 0.36
Classification Report: 
               precision    recall  f1-score   support

    negative       0.00      0.00      0.00        25
     neutral       0.00      0.00      0.00         7
    positive       0.36      1.00      0.53        18

    accuracy                           0.36        50
   macro avg       0.12      0.33      0.18        50
weighted avg       0.13      0.36      0.19        50



C:\Users\Lenovo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Lenovo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Lenovo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classific

### <h1>Logistic Regression</h1>

<h1>1. TF-IDF + Logistic Regression</h1>

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

print("\n=== PERCOBAAN 5: TF-IDF + Logistic Regression ===")

#split data
X_train, X_test, y_train, y_test = train_test_split(df['bersih'], df['label'], test_size=0.2, random_state=42)

# Model: Logistic Regression (Menggunakan fitur TF-IDF dari percobaan sebelumnya)
lr_model = LogisticRegression()
lr_model.fit(X_train_tfidf, y_train)
y_pred_lr = lr_model.predict(X_test_tfidf)

print("\n======================================")
print("Akurasi TF-IDF + Logistic Regression", accuracy_score(y_test, y_pred_lr))
print("======================================")
print("Classification Report:", "\n", classification_report(y_test, y_pred_lr))


=== PERCOBAAN 5: TF-IDF + Logistic Regression ===

Akurasi TF-IDF + Logistic Regression 0.82
Classification Report: 
               precision    recall  f1-score   support

    negative       0.92      0.88      0.90        25
     neutral       0.75      0.43      0.55         7
    positive       0.73      0.89      0.80        18

    accuracy                           0.82        50
   macro avg       0.80      0.73      0.75        50
weighted avg       0.83      0.82      0.81        50



<h1>2. Word2Vec + Logistic Regression</h1>

In [67]:
import numpy as np
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

print("\n=== PERCOBAAN 5: Word2Vec + Logistic Regression ===")

# --- 1. Training Word2Vec ---
sentences = [text.split() for text in df['bersih']]
w2v_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

def sent_vectorizer(sent, model):
    words = sent.split()
    vector_list = [model.wv[word] for word in words if word in model.wv]
    if len(vector_list) > 0:
        return np.mean(vector_list, axis=0)
    else:
        return np.zeros(model.vector_size)

X = np.array([sent_vectorizer(text, w2v_model) for text in df['bersih']])
y = df['label']

# --- 2. Klasifikasi Logistic Regression ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model Logistic Regression
lr = LogisticRegression(max_iter=1000)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)

print("\n======================================")
print("Akurasi Word2Vec + Logistic Regression", accuracy_score(y_test, y_pred))
print("======================================")
print("Classification Report:", "\n", classification_report(y_test, y_pred))


=== PERCOBAAN 5: Word2Vec + Logistic Regression ===

Akurasi Word2Vec + Logistic Regression 0.36
Classification Report: 
               precision    recall  f1-score   support

    negative       0.00      0.00      0.00        25
     neutral       0.00      0.00      0.00         7
    positive       0.36      1.00      0.53        18

    accuracy                           0.36        50
   macro avg       0.12      0.33      0.18        50
weighted avg       0.13      0.36      0.19        50



C:\Users\Lenovo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Lenovo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Lenovo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classific

### RNN

<h1>1. Word2Vec + simple RNN>

In [71]:
import numpy as np
from gensim.models import Word2Vec
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

print("\n=== PERCOBAAN 6: Word2Vec + RNN ===")

# --- 1. Persiapan Data & Word2Vec ---
# Label Encoding (Ubah label teks ke angka)
le = LabelEncoder()
y_encoded = le.fit_transform(df['label'])
num_classes = len(np.unique(y_encoded))

# Training Word2Vec
sentences = [text.split() for text in df['bersih']]
w2v_size = 100
w2v_model = Word2Vec(sentences, vector_size=w2v_size, window=5, min_count=1, workers=4)

# Tokenisasi untuk Deep Learning
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['bersih'])
X_seq = tokenizer.texts_to_sequences(df['bersih'])
vocab_size = len(tokenizer.word_index) + 1

# Padding (menyamakan panjang kalimat)
max_length = 20 
X_pad = pad_sequences(X_seq, maxlen=max_length)

# Membuat Embedding Matrix dari bobot Word2Vec
embedding_matrix = np.zeros((vocab_size, w2v_size))
for word, i in tokenizer.word_index.items():
    if word in w2v_model.wv:
        embedding_matrix[i] = w2v_model.wv[word]
    else:
        # Jika kata tidak ada di W2V, biarkan nol atau random (opsional)
        pass

# Split Data
X_train, X_test, y_train, y_test = train_test_split(X_pad, y_encoded, test_size=0.2, random_state=42)

# --- 2. Membangun Model RNN ---
model = Sequential()
# Layer Embedding menggunakan bobot dari Word2Vec (trainable=False agar bobot tidak rusak saat awal training)
model.add(Embedding(vocab_size, w2v_size, weights=[embedding_matrix], input_length=max_length, trainable=False))
model.add(SimpleRNN(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(num_classes, activation='softmax')) # Softmax untuk multi-kelas

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Training
history = model.fit(X_train, y_train, epochs=15, batch_size=32, validation_data=(X_test, y_test), verbose=1)

# Evaluasi
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Akurasi RNN: {accuracy}")

print("\n======================================")
print("Akurasi Word2Vec + RNN", accuracy)
print("======================================")


=== PERCOBAAN 6: Word2Vec + RNN ===
Epoch 1/15


C:\Users\Lenovo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - accuracy: 0.3933 - loss: 1.0903 - val_accuracy: 0.3800 - val_loss: 1.0446
Epoch 2/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.4435 - loss: 1.0962 - val_accuracy: 0.3600 - val_loss: 1.0478
Epoch 3/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.4099 - loss: 1.0867 - val_accuracy: 0.5600 - val_loss: 1.0277
Epoch 4/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.4172 - loss: 1.0830 - val_accuracy: 0.3800 - val_loss: 1.0367
Epoch 5/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.3756 - loss: 1.0744 - val_accuracy: 0.3600 - val_loss: 1.0710
Epoch 6/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.3176 - loss: 1.1033 - val_accuracy: 0.4000 - val_loss: 1.0555
Epoch 7/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.4465 - loss: 1.0544 - val_accuracy: 0.3800 - val_loss: 1.0381
Epoch 8/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.4378 - loss: 1.0658 - val_accuracy: 0.4000 - val_loss: 1.0313
Epoch 9/15
